In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
import numpy as np 
import pandas as pd 
import seaborn as sns
!pip install gensim spacy nltk
import re 
import gensim
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
data= pd.read_csv('/content/drive/My Drive/Bachelor/Cleaned Data/Sarcasm.csv') 

In [4]:
import numpy as np
max_sequence_length = np.max(data['Number of Words in tweet'])
print("max length is:", max_sequence_length)

max length is: 56


In [5]:
text = data['cleaned_tweet']
label = data['sarcasm']

In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(text.astype(str))
sequences = tokenizer.texts_to_sequences(text.astype(str))

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))


from keras_preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Found 55395 unique tokens.
Shape of data tensor: (26095, 56)


In [7]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(label))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (26095, 2)


In [8]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

# Arabert

In [12]:
!pip install bert-for-tf2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=fd727bad148441f15a5dc32eca381bce6be1a4e25222055c1e26384d69a2385f
  Stored in directory: /root/.cache/pip/wheels/ab/a4/72/df07592cea3ae06b5e846f5e52262f8b16748e829ca354b7df
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=f25e1e13da294cc8dee42ca711616aa95df1ffb7710b7c6db056949f8386ca9e
  Stored in directory: /root/.cache/pip/wheels/c7/f3/85/b8cf1d8bfe55dc2ece0f1fcd4e91d6f8fc7b59ff3fd75329e1
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=7c6340b1f407c2c21c30ee6b5be331a60ba8a5f571abf66c102896abffddfd62
  Sto

In [13]:
import numpy as np
import math
import re
import pandas as pd
import random

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt

In [14]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert
from transformers import AutoTokenizer, AutoModel

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [15]:
arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv01",do_lower_case=False)
def encode_sentence(sent):
    tokenized = arabert_tokenizer.tokenize(sent)
    return arabert_tokenizer.convert_tokens_to_ids(tokenized)

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.70M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

# BiGRU-GRU Model

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (Relu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy',get_f1])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy2= model2.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

# Activation Function Tuning 

In [ ]:
# 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (Softplus+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model9 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softplus'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model9.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model9.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions9= model9.predict(x_test)

predictions9=np.argmax(predictions9, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions9))
cm = confusion_matrix(y_test, predictions9)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions9))
print('F1 Score: %.3f' % f1_score(y_test, predictions9))
print('Precision: %.3f' % precision_score(y_test, predictions9))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 24s 22ms/step - loss: 0.5183 - accuracy: 0.8096
Epoch 2/10
612/612 [==============================] - 14s 23ms/step - loss: 0.2981 - accuracy: 0.8840
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1489 - accuracy: 0.9522
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0823 - accuracy: 0.9763
Epoch 5/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0544 - accuracy: 0.9859
Epoch 6/10
612/612 [==============================] - 14s 22ms/step - loss: 0.0447 - accuracy: 0.9882
Epoch 7/10
612/612 [==============================] - 16s 27ms/step - loss: 0.0377 - accuracy: 0.9901
Epoch 8/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0315 - accuracy: 0.9919
Epoch 9/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0286 - accuracy: 0.9918
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.720
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (softsign+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model10 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='softsign'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model10.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model10.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

prediction10= model10.predict(x_test)

prediction10=np.argmax(prediction10, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, prediction10))
cm = confusion_matrix(y_test, prediction10)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, prediction10))
print('F1 Score: %.3f' % f1_score(y_test, prediction10))
print('Precision: %.3f' % precision_score(y_test, prediction10))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 20s 22ms/step - loss: 0.4525 - accuracy: 0.8229
Epoch 2/10
612/612 [==============================] - 13s 21ms/step - loss: 0.2441 - accuracy: 0.8996
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1157 - accuracy: 0.9597
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0640 - accuracy: 0.9799
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0421 - accuracy: 0.9874
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0318 - accuracy: 0.9910
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0275 - accuracy: 0.9928
Epoch 8/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0263 - accuracy: 0.9920
Epoch 9/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0234 - accuracy: 0.9928
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.737
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (tanh+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model11.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

prediction11= model11.predict(x_test)

prediction11=np.argmax(prediction11, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, prediction11))
cm = confusion_matrix(y_test, prediction11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, prediction11))
print('F1 Score: %.3f' % f1_score(y_test, prediction11))
print('Precision: %.3f' % precision_score(y_test, prediction11))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 21ms/step - loss: 0.4506 - accuracy: 0.8230
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.2385 - accuracy: 0.9033
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1146 - accuracy: 0.9596
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0614 - accuracy: 0.9799
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0386 - accuracy: 0.9885
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0301 - accuracy: 0.9916
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0252 - accuracy: 0.9924
Epoch 8/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0231 - accuracy: 0.9932
Epoch 9/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0212 - accuracy: 0.9932
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.727
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (selu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='selu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model11.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

prediction11= model11.predict(x_test)

prediction11=np.argmax(prediction11, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, prediction11))
cm = confusion_matrix(y_test, prediction11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, prediction11))
print('F1 Score: %.3f' % f1_score(y_test, prediction11))
print('Precision: %.3f' % precision_score(y_test, prediction11))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 22ms/step - loss: 0.4506 - accuracy: 0.8215
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.2381 - accuracy: 0.9034
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1097 - accuracy: 0.9605
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0564 - accuracy: 0.9824
Epoch 5/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0356 - accuracy: 0.9893
Epoch 6/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0274 - accuracy: 0.9925
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0242 - accuracy: 0.9926
Epoch 8/10
612/612 [==============================] - 14s 24ms/step - loss: 0.0221 - accuracy: 0.9934
Epoch 9/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0208 - accuracy: 0.9927
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.713
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (elu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model12 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model12.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model12.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

prediction12= model12.predict(x_test)

prediction12=np.argmax(prediction12, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, prediction12))
cm = confusion_matrix(y_test, prediction12)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, prediction12))
print('F1 Score: %.3f' % f1_score(y_test, prediction12))
print('Precision: %.3f' % precision_score(y_test, prediction12))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 22ms/step - loss: 0.4473 - accuracy: 0.8237
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.2363 - accuracy: 0.9022
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1139 - accuracy: 0.9589
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0600 - accuracy: 0.9812
Epoch 5/10
612/612 [==============================] - 16s 26ms/step - loss: 0.0408 - accuracy: 0.9878
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0315 - accuracy: 0.9914
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0266 - accuracy: 0.9926
Epoch 8/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0243 - accuracy: 0.9924
Epoch 9/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0225 - accuracy: 0.9929
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.762
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), (exponential+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='exponential'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model13.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

prediction13= model13.predict(x_test)

prediction13=np.argmax(prediction13, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, prediction13))
cm = confusion_matrix(y_test, prediction13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, prediction13))
print('F1 Score: %.3f' % f1_score(y_test, prediction13))
print('Precision: %.3f' % precision_score(y_test, prediction13))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 22ms/step - loss: 0.5989 - accuracy: 0.7743
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.3191 - accuracy: 0.8786
Epoch 3/10
612/612 [==============================] - 13s 22ms/step - loss: 0.1507 - accuracy: 0.9511
Epoch 4/10
612/612 [==============================] - 15s 24ms/step - loss: 0.0863 - accuracy: 0.9744
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0539 - accuracy: 0.9845
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0424 - accuracy: 0.9880
Epoch 7/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0324 - accuracy: 0.9913
Epoch 8/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0288 - accuracy: 0.9920
Epoch 9/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0282 - accuracy: 0.9922
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.699
The confusio

# Learning Rate & Optimizers Tuning on (Relu + Softmax)

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-3, Adam

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 21ms/step - loss: 0.3723 - accuracy: 0.8469
Epoch 2/10
612/612 [==============================] - 14s 23ms/step - loss: 0.1389 - accuracy: 0.9506
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0422 - accuracy: 0.9857
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0199 - accuracy: 0.9913
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0167 - accuracy: 0.9928
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0124 - accuracy: 0.9942
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0129 - accuracy: 0.9938
Epoch 8/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0131 - accuracy: 0.9929
Epoch 9/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0114 - accuracy: 0.9941
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.706
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-2, Adam

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 21s 21ms/step - loss: 0.4115 - accuracy: 0.8274
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.2689 - accuracy: 0.8949
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.1866 - accuracy: 0.9317
Epoch 4/10
612/612 [==============================] - 13s 21ms/step - loss: 0.1398 - accuracy: 0.9487
Epoch 5/10
612/612 [==============================] - 13s 21ms/step - loss: 0.1167 - accuracy: 0.9593
Epoch 6/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0998 - accuracy: 0.9645
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0912 - accuracy: 0.9697
Epoch 8/10
612/612 [==============================] - 13s 22ms/step - loss: 0.0796 - accuracy: 0.9719
Epoch 9/10
612/612 [==============================] - 13s 21ms/step - loss: 0.0723 - accuracy: 0.9757
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.640
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-1, Adam

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-1),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 21ms/step - loss: 0.4948 - accuracy: 0.8171
Epoch 2/10
612/612 [==============================] - 13s 22ms/step - loss: 0.4726 - accuracy: 0.8206
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.4725 - accuracy: 0.8206
Epoch 4/10
612/612 [==============================] - 13s 22ms/step - loss: 0.4723 - accuracy: 0.8206
Epoch 5/10
612/612 [==============================] - 13s 22ms/step - loss: 0.4722 - accuracy: 0.8206
Epoch 6/10
612/612 [==============================] - 13s 21ms/step - loss: 0.4730 - accuracy: 0.8206
Epoch 7/10
612/612 [==============================] - 13s 22ms/step - loss: 0.4723 - accuracy: 0.8206
Epoch 8/10
612/612 [==============================] - 15s 24ms/step - loss: 0.4725 - accuracy: 0.8206
Epoch 9/10
612/612 [==============================] - 16s 25ms/step - loss: 0.4722 - accuracy: 0.8206
Epoch 10/10
204/204 [==============================] - 4s 11ms/step
Recall: 0.000
The confusi

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-4, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 19ms/step - loss: 0.4771 - accuracy: 0.8208
Epoch 2/10
612/612 [==============================] - 11s 19ms/step - loss: 0.3992 - accuracy: 0.8289
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.3384 - accuracy: 0.8592
Epoch 4/10
612/612 [==============================] - 11s 19ms/step - loss: 0.3096 - accuracy: 0.8742
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2844 - accuracy: 0.8873
Epoch 6/10
612/612 [==============================] - 11s 19ms/step - loss: 0.2620 - accuracy: 0.8980
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2406 - accuracy: 0.9078
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2183 - accuracy: 0.9181
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.1953 - accuracy: 0.9272
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.602
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-3, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-3),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 19ms/step - loss: 0.3978 - accuracy: 0.8333
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.2439 - accuracy: 0.9058
Epoch 3/10
612/612 [==============================] - 12s 20ms/step - loss: 0.1566 - accuracy: 0.9440
Epoch 4/10
612/612 [==============================] - 12s 19ms/step - loss: 0.1114 - accuracy: 0.9602
Epoch 5/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0847 - accuracy: 0.9711
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0653 - accuracy: 0.9786
Epoch 7/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0506 - accuracy: 0.9839
Epoch 8/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0427 - accuracy: 0.9873
Epoch 9/10
612/612 [==============================] - 11s 19ms/step - loss: 0.0340 - accuracy: 0.9891
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.635
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-2, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-2),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 19ms/step - loss: 0.3973 - accuracy: 0.8422
Epoch 2/10
612/612 [==============================] - 11s 19ms/step - loss: 0.2084 - accuracy: 0.9290
Epoch 3/10
612/612 [==============================] - 13s 21ms/step - loss: 0.1352 - accuracy: 0.9595
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0967 - accuracy: 0.9727
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0794 - accuracy: 0.9803
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0669 - accuracy: 0.9837
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0636 - accuracy: 0.9855
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0533 - accuracy: 0.9866
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.0465 - accuracy: 0.9891
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.720
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-1, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-1),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 18ms/step - loss: 0.8676 - accuracy: 0.7958
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5522 - accuracy: 0.8179
Epoch 3/10
612/612 [==============================] - 12s 20ms/step - loss: 0.5713 - accuracy: 0.8183
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5734 - accuracy: 0.8158
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5593 - accuracy: 0.8185
Epoch 6/10
612/612 [==============================] - 11s 19ms/step - loss: 0.5661 - accuracy: 0.8190
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5612 - accuracy: 0.8182
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5643 - accuracy: 0.8184
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5541 - accuracy: 0.8193
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.000
The confusio

In [ ]:
#Model 3: 1 Bi-directional GRU - Bidirectional LSTM, 1e-1, RMSprop

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-1),
              metrics=['accuracy'])

model2.fit(x_train, y_train, batch_size=32, epochs=10)

# accuracy2= model2.evaluate(x_test, y_test)
# # print('test loss is:', loss)
# print('test accuracy is: ', accuracy2)

predictions2= model2.predict(x_test)

predictions2=np.argmax(predictions2, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions2))
cm = confusion_matrix(y_test, predictions2)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions2))
print('F1 Score: %.3f' % f1_score(y_test, predictions2))
print('Precision: %.3f' % precision_score(y_test, predictions2))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 17s 18ms/step - loss: 0.8676 - accuracy: 0.7958
Epoch 2/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5522 - accuracy: 0.8179
Epoch 3/10
612/612 [==============================] - 12s 20ms/step - loss: 0.5713 - accuracy: 0.8183
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5734 - accuracy: 0.8158
Epoch 5/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5593 - accuracy: 0.8185
Epoch 6/10
612/612 [==============================] - 11s 19ms/step - loss: 0.5661 - accuracy: 0.8190
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5612 - accuracy: 0.8182
Epoch 8/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5643 - accuracy: 0.8184
Epoch 9/10
612/612 [==============================] - 11s 18ms/step - loss: 0.5541 - accuracy: 0.8193
Epoch 10/10
204/204 [==============================] - 3s 7ms/step
Recall: 0.000
The confusio

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Nadam(1e-4), (Relu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Nadam(1e-4),
              metrics=['accuracy'])

model16.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy16= model16.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy16)

predictions16= model16.predict(x_test)

predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions16))
cm = confusion_matrix(y_test, predictions16)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))

Epoch 1/10
612/612 [==============================] - 185s 268ms/step - loss: 0.4677 - accuracy: 0.8199
Epoch 2/10
612/612 [==============================] - 134s 220ms/step - loss: 0.2860 - accuracy: 0.8813
Epoch 3/10
612/612 [==============================] - 127s 207ms/step - loss: 0.1620 - accuracy: 0.9426
Epoch 4/10
612/612 [==============================] - 128s 209ms/step - loss: 0.1000 - accuracy: 0.9677
Epoch 5/10
612/612 [==============================] - 129s 211ms/step - loss: 0.0648 - accuracy: 0.9802
Epoch 6/10
612/612 [==============================] - 132s 216ms/step - loss: 0.0461 - accuracy: 0.9866
Epoch 7/10
612/612 [==============================] - 131s 213ms/step - loss: 0.0341 - accuracy: 0.9900
Epoch 8/10
612/612 [==============================] - 128s 209ms/step - loss: 0.0305 - accuracy: 0.9909
Epoch 9/10
612/612 [==============================] - 130s 213ms/step - loss: 0.0254 - accuracy: 0.9919
Epoch 10/10
204/204 [==============================] - 8s 31ms/s

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [17]:
#Bi-directional GRU - Bidirectional LSTM, Adagrad(1e-4), (Relu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model17 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model17.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adagrad(1e-4),
              metrics=['accuracy'])

model17.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy17= model17.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy17)

predictions17= model17.predict(x_test)

predictions17=np.argmax(predictions17, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions17))
cm = confusion_matrix(y_test, predictions17)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions17))
print('F1 Score: %.3f' % f1_score(y_test, predictions17))
print('Precision: %.3f' % precision_score(y_test, predictions17))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 15ms/step - loss: 0.6852 - accuracy: 0.7885
Epoch 2/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6744 - accuracy: 0.8216
Epoch 3/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6651 - accuracy: 0.8216
Epoch 4/10
612/612 [==============================] - 11s 18ms/step - loss: 0.6565 - accuracy: 0.8216
Epoch 5/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6482 - accuracy: 0.8216
Epoch 6/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6400 - accuracy: 0.8216
Epoch 7/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6314 - accuracy: 0.8216
Epoch 8/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6222 - accuracy: 0.8216
Epoch 9/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6131 - accuracy: 0.8216
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.5989 - accuracy: 0.819

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [20]:
#Bi-directional GRU - Bidirectional LSTM, Adadelta(1e-4), (Relu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adadelta(1e-4),
              metrics=['accuracy'])

model18.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy18= model18.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy18)

predictions18= model18.predict(x_test)

predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions18))
cm = confusion_matrix(y_test, predictions18)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))

Epoch 1/10
612/612 [==============================] - 15s 15ms/step - loss: 0.6906 - accuracy: 0.6620
Epoch 2/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6896 - accuracy: 0.7135
Epoch 3/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6885 - accuracy: 0.7513
Epoch 4/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6875 - accuracy: 0.7813
Epoch 5/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6863 - accuracy: 0.7990
Epoch 6/10
612/612 [==============================] - 11s 18ms/step - loss: 0.6853 - accuracy: 0.8095
Epoch 7/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6841 - accuracy: 0.8149
Epoch 8/10
612/612 [==============================] - 10s 16ms/step - loss: 0.6829 - accuracy: 0.8198
Epoch 9/10
612/612 [==============================] - 9s 15ms/step - loss: 0.6817 - accuracy: 0.8213
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.6802 - a

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [22]:
#Bi-directional GRU - Bidirectional LSTM, adamax(1e-4), (Relu+softmax)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model18 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model18.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adamax(1e-4),
              metrics=['accuracy'])

model18.fit(x_train, y_train, batch_size=32, epochs=10)

accuracy18= model18.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy18)

predictions18= model18.predict(x_test)

predictions18=np.argmax(predictions18, axis=1)
y_test=np.argmax(y_test, axis=1)

print('Recall: %.3f' % recall_score(y_test, predictions18))
cm = confusion_matrix(y_test, predictions18)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions18))
print('F1 Score: %.3f' % f1_score(y_test, predictions18))
print('Precision: %.3f' % precision_score(y_test, predictions18))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 15s 16ms/step - loss: 0.4981 - accuracy: 0.8177
Epoch 2/10
612/612 [==============================] - 10s 16ms/step - loss: 0.4602 - accuracy: 0.8210
Epoch 3/10
612/612 [==============================] - 10s 17ms/step - loss: 0.4546 - accuracy: 0.8210
Epoch 4/10
612/612 [==============================] - 10s 16ms/step - loss: 0.4404 - accuracy: 0.8210
Epoch 5/10
612/612 [==============================] - 10s 16ms/step - loss: 0.4215 - accuracy: 0.8223
Epoch 6/10
612/612 [==============================] - 10s 17ms/step - loss: 0.3889 - accuracy: 0.8307
Epoch 7/10
612/612 [==============================] - 11s 18ms/step - loss: 0.3561 - accuracy: 0.8410
Epoch 8/10
612/612 [==============================] - 10s 16ms/step - loss: 0.3335 - accuracy: 0.8510
Epoch 9/10
612/612 [==============================] - 11s 19ms/step - loss: 0.3148 - accuracy: 0.8620
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.3746 - accuracy:

# Batch Size Tuning

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Batch(10)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model10 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model10.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model10.fit(x_train, y_train, batch_size=10, epochs=10)

accuracy10= model10.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy10)

predictions10= model10.predict(x_test)

predictions10=np.argmax(predictions10, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions10)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions10))
print('Recall: %.3f' % recall_score(y_test, predictions10))
print('Precision: %.3f' % precision_score(y_test, predictions10))
print('F1 Score: %.3f' % f1_score(y_test, predictions10))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1958/1958 [==============================] - 73s 29ms/step - loss: 0.4142 - accuracy: 0.8320
Epoch 2/10
1958/1958 [==============================] - 52s 27ms/step - loss: 0.2103 - accuracy: 0.9217
Epoch 3/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.1082 - accuracy: 0.9641
Epoch 4/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.0614 - accuracy: 0.9816
Epoch 5/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.0415 - accuracy: 0.9877
Epoch 6/10
1958/1958 [==============================] - 40s 21ms/step - loss: 0.0303 - accuracy: 0.9915
Epoch 7/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.0261 - accuracy: 0.9921
Epoch 8/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.0185 - accuracy: 0.9933
Epoch 9/10
1958/1958 [==============================] - 39s 20ms/step - loss: 0.0142 - accuracy: 0.9936
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Batch(40)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model11 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model11.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model11.fit(x_train, y_train, batch_size=40, epochs=10)

accuracy11= model11.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy11)

predictions11= model11.predict(x_test)

predictions11=np.argmax(predictions11, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions11)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions11))
print('Recall: %.3f' % recall_score(y_test, predictions11))
print('Precision: %.3f' % precision_score(y_test, predictions11))
print('F1 Score: %.3f' % f1_score(y_test, predictions11))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


490/490 [==============================] - 15s 21ms/step - loss: 0.4750 - accuracy: 0.8195
Epoch 2/10
490/490 [==============================] - 10s 20ms/step - loss: 0.2750 - accuracy: 0.8855
Epoch 3/10
490/490 [==============================] - 10s 20ms/step - loss: 0.1356 - accuracy: 0.9526
Epoch 4/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0752 - accuracy: 0.9759
Epoch 5/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0473 - accuracy: 0.9867
Epoch 6/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0365 - accuracy: 0.9906
Epoch 7/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0306 - accuracy: 0.9919
Epoch 8/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0299 - accuracy: 0.9921
Epoch 9/10
490/490 [==============================] - 10s 20ms/step - loss: 0.0269 - accuracy: 0.9927
Epoch 10/10
204/204 [==============================] - 4s 11ms/step - loss: 0.4579 - accuracy

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Batch(60)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model12 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model12.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model12.fit(x_train, y_train, batch_size=60, epochs=10)

accuracy12= model12.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy12)

predictions12= model12.predict(x_test)

predictions12=np.argmax(predictions12, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions12)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions12))
print('Recall: %.3f' % recall_score(y_test, predictions12))
print('Precision: %.3f' % precision_score(y_test, predictions12))
print('F1 Score: %.3f' % f1_score(y_test, predictions12))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


327/327 [==============================] - 14s 21ms/step - loss: 0.4959 - accuracy: 0.8109
Epoch 2/10
327/327 [==============================] - 8s 24ms/step - loss: 0.3336 - accuracy: 0.8562
Epoch 3/10
327/327 [==============================] - 7s 20ms/step - loss: 0.1625 - accuracy: 0.9392
Epoch 4/10
327/327 [==============================] - 7s 21ms/step - loss: 0.0861 - accuracy: 0.9725
Epoch 5/10
327/327 [==============================] - 7s 20ms/step - loss: 0.0512 - accuracy: 0.9857
Epoch 6/10
327/327 [==============================] - 7s 21ms/step - loss: 0.0380 - accuracy: 0.9900
Epoch 7/10
327/327 [==============================] - 7s 20ms/step - loss: 0.0320 - accuracy: 0.9923
Epoch 8/10
327/327 [==============================] - 7s 20ms/step - loss: 0.0271 - accuracy: 0.9930
Epoch 9/10
327/327 [==============================] - 7s 20ms/step - loss: 0.0251 - accuracy: 0.9933
Epoch 10/10
204/204 [==============================] - 4s 12ms/step - loss: 0.4577 - accuracy: 0.8991

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Batch(80)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model16 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model16.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model16.fit(x_train, y_train, batch_size=80, epochs=10)

accuracy16= model16.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy16)

predictions16= model16.predict(x_test)

predictions16=np.argmax(predictions16, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions16)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions16))
print('Recall: %.3f' % recall_score(y_test, predictions16))
print('Precision: %.3f' % precision_score(y_test, predictions16))
print('F1 Score: %.3f' % f1_score(y_test, predictions16))

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


245/245 [==============================] - 19s 31ms/step - loss: 0.5146 - accuracy: 0.8109
Epoch 2/10
245/245 [==============================] - 9s 35ms/step - loss: 0.3782 - accuracy: 0.8410
Epoch 3/10
245/245 [==============================] - 8s 34ms/step - loss: 0.1927 - accuracy: 0.9284
Epoch 4/10
245/245 [==============================] - 6s 24ms/step - loss: 0.1053 - accuracy: 0.9660
Epoch 5/10
245/245 [==============================] - 5s 22ms/step - loss: 0.0635 - accuracy: 0.9826
Epoch 6/10
245/245 [==============================] - 5s 22ms/step - loss: 0.0428 - accuracy: 0.9890
Epoch 7/10
245/245 [==============================] - 5s 22ms/step - loss: 0.0345 - accuracy: 0.9912
Epoch 8/10
245/245 [==============================] - 5s 22ms/step - loss: 0.0294 - accuracy: 0.9929
Epoch 9/10
245/245 [==============================] - 6s 23ms/step - loss: 0.0267 - accuracy: 0.9927
Epoch 10/10
204/204 [==============================] - 3s 8ms/step - loss: 0.4628 - accuracy: 0.9045


# Epochs Tuning

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Epochs (25)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model13 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model13.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model13.fit(x_train, y_train, batch_size=32, epochs=25)

accuracy13= model13.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy13)

predictions13= model13.predict(x_test)

predictions13=np.argmax(predictions13, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions13)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions13))
print('Recall: %.3f' % recall_score(y_test, predictions13))
print('Precision: %.3f' % precision_score(y_test, predictions13))
print('F1 Score: %.3f' % f1_score(y_test, predictions13))

Epoch 1/25


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 24s 28ms/step - loss: 0.4631 - accuracy: 0.8204
Epoch 2/25
612/612 [==============================] - 18s 29ms/step - loss: 0.2562 - accuracy: 0.8984
Epoch 3/25
612/612 [==============================] - 17s 28ms/step - loss: 0.1284 - accuracy: 0.9553
Epoch 4/25
612/612 [==============================] - 17s 28ms/step - loss: 0.0735 - accuracy: 0.9777
Epoch 5/25
612/612 [==============================] - 17s 29ms/step - loss: 0.0485 - accuracy: 0.9874
Epoch 6/25
612/612 [==============================] - 14s 23ms/step - loss: 0.0382 - accuracy: 0.9904
Epoch 7/25
612/612 [==============================] - 12s 20ms/step - loss: 0.0330 - accuracy: 0.9916
Epoch 8/25
612/612 [==============================] - 12s 20ms/step - loss: 0.0305 - accuracy: 0.9917
Epoch 9/25
612/612 [==============================] - 13s 20ms/step - loss: 0.0271 - accuracy: 0.9924
Epoch 10/25
612/612 [==============================] - 13s 21ms/step - loss: 0.0240 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Epochs (50)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model14 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model14.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model14.fit(x_train, y_train, batch_size=32, epochs=50)

accuracy14= model14.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy14)

predictions14= model14.predict(x_test)

predictions14=np.argmax(predictions14, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions14)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions14))
print('Recall: %.3f' % recall_score(y_test, predictions14))
print('Precision: %.3f' % precision_score(y_test, predictions14))
print('F1 Score: %.3f' % f1_score(y_test, predictions14))

Epoch 1/50


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 18s 21ms/step - loss: 0.4690 - accuracy: 0.8176
Epoch 2/50
612/612 [==============================] - 13s 20ms/step - loss: 0.2691 - accuracy: 0.8909
Epoch 3/50
612/612 [==============================] - 13s 21ms/step - loss: 0.1350 - accuracy: 0.9553
Epoch 4/50
612/612 [==============================] - 13s 21ms/step - loss: 0.0764 - accuracy: 0.9773
Epoch 5/50
612/612 [==============================] - 13s 21ms/step - loss: 0.0519 - accuracy: 0.9854
Epoch 6/50
612/612 [==============================] - 13s 21ms/step - loss: 0.0378 - accuracy: 0.9900
Epoch 7/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0331 - accuracy: 0.9914
Epoch 8/50
612/612 [==============================] - 14s 23ms/step - loss: 0.0276 - accuracy: 0.9926
Epoch 9/50
612/612 [==============================] - 15s 25ms/step - loss: 0.0244 - accuracy: 0.9932
Epoch 10/50
612/612 [==============================] - 15s 24ms/step - loss: 0.0228 - accurac

In [ ]:
# #shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

# split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [ ]:
#Bi-directional GRU - Bidirectional LSTM, Adam(1e-4), Epochs (75)

VOCAB_SIZE = len(arabert_tokenizer.get_vocab())
EMB_DIM = 200

from tensorflow.keras import layers
model15 = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, EMB_DIM),
    tf.keras.layers.Bidirectional(layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(EMB_DIM, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
])


model15.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model15.fit(x_train, y_train, batch_size=32, epochs=75)

accuracy15= model15.evaluate(x_test, y_test)
# print('test loss is:', loss)
print('test accuracy is: ', accuracy15)

predictions15= model15.predict(x_test)

predictions15=np.argmax(predictions15, axis=1)
y_test=np.argmax(y_test, axis=1)


cm = confusion_matrix(y_test, predictions15)
print('The confusion matrix is', cm)
print('Accuracy: %.3f' % accuracy_score(y_test, predictions15))
print('Recall: %.3f' % recall_score(y_test, predictions15))
print('Precision: %.3f' % precision_score(y_test, predictions15))
print('F1 Score: %.3f' % f1_score(y_test, predictions15))

Epoch 1/75


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


612/612 [==============================] - 19s 22ms/step - loss: 0.4648 - accuracy: 0.8171
Epoch 2/75
612/612 [==============================] - 13s 22ms/step - loss: 0.2567 - accuracy: 0.8998
Epoch 3/75
612/612 [==============================] - 14s 22ms/step - loss: 0.1275 - accuracy: 0.9565
Epoch 4/75
612/612 [==============================] - 14s 24ms/step - loss: 0.0741 - accuracy: 0.9767
Epoch 5/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0494 - accuracy: 0.9862
Epoch 6/75
612/612 [==============================] - 13s 22ms/step - loss: 0.0364 - accuracy: 0.9900
Epoch 7/75
612/612 [==============================] - 13s 22ms/step - loss: 0.0315 - accuracy: 0.9910
Epoch 8/75
612/612 [==============================] - 16s 27ms/step - loss: 0.0280 - accuracy: 0.9919
Epoch 9/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0242 - accuracy: 0.9929
Epoch 10/75
612/612 [==============================] - 14s 22ms/step - loss: 0.0221 - accurac